In [ ]:
# The code for generating the topology file of polymers is created by Xueying Yuan and Xian Kong from School of Emergent Soft Matter, South China University of Technology, China
# E-mail: xiankong@icloud.com and xueyingyuan628@gmail.com
# The information contained in this file is provided for reference purposes only and does not constitute any advice or guarantee. The author does not assume responsibility for the accuracy of the referenced information, and readers are advised to verify it independently. 


In [ ]:
import numpy as np
import random
import mbuild as mb
from mbuild.lib.recipes.polymer import Polymer
from mbuild.coordinate_transform import force_overlap
from mbuild.lib.atoms import H
from mbuild.port import Port
from foyer import Forcefield
from openbabel import openbabel as ob

In [ ]:
monomers = 'CCO'  # Example SMILES for a monomer unit
block2_smiles = 'C'  # SMILES for the end group
repeat_units = 50  # Number of times the monomer unit is repeated

In [ ]:
# Generate the .itp file from the .top file
def getItpFromTop(top, itp, molname):
    print("Processing", top)
    with open(top, "r") as f:
        lines = [line.rstrip('\n') for line in f]
    nLmoleculetype, nLatoms, nLbonds, nLsystem = -1, -1, -1, -1

    for i, line in enumerate(lines):
        if "[ moleculetype ]" in line:
            nLmoleculetype = i
        elif "[ atoms ]" in line:
            nLatoms = i
        elif "[ bonds ]" in line:
            nLbonds = i
        elif "[ system ]" in line:
            nLsystem = i
    # Check that all required sections are found
    if -1 in (nLmoleculetype, nLatoms, nLbonds, nLsystem):
        raise ValueError("One or more required sections missing in the .top file!")

    print(f"Located sections: moleculetype={nLmoleculetype}, atoms={nLatoms}, bonds={nLbonds}, system={nLsystem}")

    lines[nLmoleculetype + 2] = f"{molname}          3"
    # Write the modified content into the new .itp file
    with open(itp, "w") as f:
        atmidx = 0
        for i, line in enumerate(lines[nLmoleculetype:nLsystem]):
            if nLatoms < i + nLmoleculetype < nLbonds:
                if len(line.strip()) > 0 and not line.startswith(";"):
                    dat = line.split()
                    if dat[3] == "RES":
                        dat[3] = molname
                    line = " ".join(dat[:8]) 
                    atmidx += 1
            f.write(line + '\n')

    print("ITP file generation complete.")

In [ ]:
# Construct the polymer SMILES string by repeating the monomer and adding the end group
polymer_smiles = monomers * repeat_units + block2_smiles
mol = mb.load(polymer_smiles, smiles=True, backend='pybel')


In [ ]:
# Load the oplsaa force field
ff = Forcefield(forcefield_files='./oplsaaXian.xml')
obConversion = ob.OBConversion()
mol_ob = ob.OBMol()
ob_method = "eem"  #Set the charge calculation method to EEM
ob_charge_model = ob.OBChargeModel.FindType(ob_method)
mol.save('./mol.mol2', overwrite=True)
mol.save('./mol.pdb', overwrite=True)
mol_top = ff.apply(mol, verbose=True)

In [ ]:
# Load the PDB file into Open Babel and compute partial charges
obConversion.ReadFile(mol_ob, './mol.pdb')
ob_charge_model.ComputeCharges(mol_ob)
charges = ob_charge_model.GetPartialCharges()

In [ ]:
# Assign partial charges to the topology and calculate total charge
total_charge_sum = 0
for atom, charge in zip(mol_top.atoms, charges):
    atom.charge = charge
    total_charge_sum += charge
print(f"Total charge sum: {total_charge_sum}")

In [ ]:
# Save the topology file
mol_top.save('./mol.top', overwrite=True)
molname = "mol"
getItpFromTop('./mol.top', './mol.itp', molname)